# Trabalho 1 - Nivelamento

Considere os datasets abaixo, estratégias de pré-processamento, medidas de avaliação, métodos de comparação estatística e os seguintes algoritmos de aprendizado de máquina: árvore de decisão, random forest e k-nearest neighbor. A partir disso, responda as seguintes perguntas:

1. Qual o algoritmo de AM mais adequado para cada problema?
2. Qual o algoritmo de AM mais adequado para todos os problemas?
knn:85.94066985255232
Arvore de decisão:85.42638695604664
Random Florest:86.62282691074151
Partindo de uma comparação direta entre os resultados do cross validation usando o msm conjunto de dados para todos, o algoritmo de random florest é o melhor, no entanto é o mais demorado também demorando 31 minutos para processar tudo, enquanto so demais demoraram apenas 1 minuto cada, em segundo fica o knn e terceiro a arvore de decisão que tem tempos similares. As únicas alterações feitas no dataset foi a seleção de atributos, e a conversão de atributos nominais para numéricos/binários utilizando o getdummies ou a conversão direta para os que tinham apenas duas classes.

Para responder essas questões construa um notebook no colab ou um ambiente similar. Documente de forma clara cada passo e justifique suas decisões.

In [1]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import matplotlib.patches as mpatches
import matplotlib as plt
from scipy.io import arff
import urllib.request
import pandas as pd
import numpy as np
import io

In [2]:
'''Link com os datasets'''
PhishingWebsites_link = "https://www.openml.org/data/download/1798106/phpV5QYya"
arrhythmia_link       = "https://www.openml.org/data/download/53551/arrhythmia.arff"
Satellite_link        = "https://www.openml.org/data/download/16787463/phpZrCzJR"
airlines_link         = "https://www.openml.org/data/download/66526/phpvcoG8S"
AedesSex_link         = "https://github.com/denismr/Classification-and-Counting-with-Recurrent-Contexts/raw/master/codeAndData/data/AedesSex.csv"
phoneme_link          = "https://www.openml.org/data/download/1592281/php8Mz7BG"
adult_link            = "https://www.openml.org/data/download/1595261/phpMawTba"

In [3]:
def loadDataset_Arff(url: str) -> pd.DataFrame:
  '''Função responsável por ler os datasets do tipo .arff'''
  ftpstream = urllib.request.urlopen(url)
  return pd.DataFrame(arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))[0])
  
def get_dummies(df: pd.DataFrame, col:str) -> pd.DataFrame:
  '''Função responsável por separar os dados nominais em outras colunas.'''
  return pd.get_dummies(df,prefix=col,prefix_sep='.',columns=[col]).copy()

In [4]:
def proportion(Dataset,target="Target"):
  '''Função que gera um gráfico de proporção dos valores targets'''
  target_count = Dataset[target].value_counts()
  print('Class 0:', target_count[0])
  print('Class 1:', target_count[1])
  target_count.plot(kind='bar', title='Count (target)');
  
def normalize(dataset,target='Target') -> pd.DataFrame:
  '''Função que retorna o dataset normalizado'''
  X = dataset.drop([target], axis=1).copy()
  Y = dataset[target].copy()

  scaler = MinMaxScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
  dataset = X
  dataset[target] = Y
  return dataset

In [5]:
def knn(dataset,n_neighbors=3,target='Target'):
  '''Função que aplica o knn'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = KNeighborsClassifier(n_neighbors=3)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def treeDecision(dataset,target='Target'):
  '''Função que aplica a arvore de decisão'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = DecisionTreeClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def RForestDecision(dataset,target='Target'):
  '''Função que aplica random florest'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)
  

In [25]:
def crossVal(x_Test ,y_Test,x_train,y_train):
  '''Função que aplica o knn com x e y previamente setados'''
  modelDT = DecisionTreeClassifier()
  modelDT.fit(x_train,y_train)
  modeRF = RandomForestClassifier()
  modeRF.fit(x_train,y_train)
  modelKNN = KNeighborsClassifier(n_neighbors=3)
  modelKNN.fit(x_train,y_train)

  return cross_val_score(modelDT, x_Test, y_Test, cv=10, scoring="f1"),cross_val_score(modeRF, x_Test, y_Test, cv=10, scoring="f1"),cross_val_score(modelKNN, x_Test, y_Test, cv=10, scoring="f1")

In [8]:
def selectAttributes(dataset):
  '''Função que aplica o knn com x e y previamente setados'''
  model = DecisionTreeClassifier()#max_leaf_nodes=10
  feature_ = RFE(model,n_features_to_select=5, step=1)
  return feature_.fit(dataset.drop(['Target'],axis=1),dataset['Target']).support_

def removeColumns(dataset):
  '''Função que faz a seleção de atributos e retorna uma lista com o nome das colunas que serão removidas'''
  attributes = selectAttributes(dataset)
  dropListIndex = np.where(attributes == False)[0]
  dropList = []
  for index in dropListIndex:
    if(dataset.columns[index] != "Target"):
      dropList.append(dataset.columns[index])
  return dropList


In [9]:
def underSampling(dataset):
  # CONTAR CLASSES
  count_class_1, count_class_0 = dataset.Target.value_counts()
  # # Divide by class
  df_class_0 = dataset[dataset['Target'] == 0]
  df_class_1 = dataset[dataset['Target'] == 1]
  df_class_1_under = df_class_1.sample(int(count_class_1*0.8))
  df_test_under = pd.concat([df_class_1_under, df_class_0], axis=0)
  return df_test_under

def overSampling(dataset):
  # CONTAR CLASSES
  count_class_1, count_class_0 = dataset.Target.value_counts()
  print(count_class_1, count_class_0)
  # # Divide by class
  df_class_0 = dataset[dataset['Target'] == 0]
  df_class_1 = dataset[dataset['Target'] == 1]
  df_class_0_over = df_class_0.sample(int(count_class_0/0.85),replace=True)
  df_test_over = pd.concat([df_class_0_over, df_class_1], axis=0)
  return df_test_over

# Lendo todos os datasets

In [10]:
InitDataset_PhishingWebsites = loadDataset_Arff(PhishingWebsites_link)
InitDataset_arrhythmia = loadDataset_Arff(arrhythmia_link)
InitDataset_Satellite = loadDataset_Arff(Satellite_link)
InitDataset_airlines = loadDataset_Arff(airlines_link)
InitDataset_AedesSex = pd.read_csv(AedesSex_link)
InitDataset_phoneme = loadDataset_Arff(phoneme_link)
InitDataset_adult = loadDataset_Arff(adult_link)

Abaixo será feito a conversão das classes targets para True e False, e trocado o nome da coluna para Target para as que não estão. Também será feito uma analise de distribuição e procura de dados nominais.

In [11]:
Dataset_PhishingWebsites = InitDataset_PhishingWebsites.copy()
Dataset_PhishingWebsites['Target'] = [True if x == '-1'.encode() else False for x in Dataset_PhishingWebsites['Result']]
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(['Result'],axis=1)
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(removeColumns(Dataset_PhishingWebsites),axis=1)

In [12]:
Dataset_arrhythmia = InitDataset_arrhythmia.copy()
Dataset_arrhythmia['Target'] = [True if x == 'P'.encode() else False for x in Dataset_arrhythmia['binaryClass']]
Dataset_arrhythmia = Dataset_arrhythmia.drop(['binaryClass'],axis=1) #Remoção da coluna J por ter 98% dos dados NaN
Dataset_arrhythmia = Dataset_arrhythmia.dropna() #Remoção das linhas com valores faltantes
Dataset_arrhythmia = Dataset_arrhythmia.drop(removeColumns(Dataset_arrhythmia),axis=1) 
Dataset_arrhythmia.reset_index(inplace=True, drop=True)

In [13]:
##Sem nominais usando a função select_nominals
Dataset_Satellite = InitDataset_Satellite.copy()
Dataset_Satellite['Target'] = [True if x == 'Anomaly'.encode() else False for x in Dataset_Satellite['Target']]
Dataset_Satellite = Dataset_Satellite.drop(removeColumns(Dataset_Satellite),axis=1) 

In [14]:
##Dataset com muitos dados nominais, para aplicar o knn será necessário usar a função getdummies em AirportFrom e AirportTo.
Dataset_airlines = InitDataset_airlines.copy()
Dataset_airlines['Target'] = [True if x == '1'.encode() else False for x in Dataset_airlines['Delay']]
Dataset_airlines = Dataset_airlines.drop(['Delay'],axis=1)

Dataset_airlines = Dataset_airlines.drop_duplicates()


Dataset_airlines['Airline'] = Dataset_airlines['Airline'].astype('category')
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].astype('category')
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].astype('category')

Dataset_airlines['Airline'] = Dataset_airlines['Airline'].cat.codes
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].cat.codes
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].cat.codes

Dataset_airlines = Dataset_airlines.drop(removeColumns(Dataset_airlines),axis=1)
Dataset_airlines.reset_index(inplace=True, drop=True)

In [15]:
Dataset_AedesSex = InitDataset_AedesSex.copy()
Dataset_AedesSex['Target'] = [True if x == 'F' else False for x in Dataset_AedesSex['sex']]
Dataset_AedesSex = Dataset_AedesSex.drop(['sex'],axis=1)
Dataset_AedesSex = Dataset_AedesSex.drop(removeColumns(Dataset_AedesSex),axis=1) 

In [16]:
Dataset_phoneme = InitDataset_phoneme.copy()
Dataset_phoneme['Target'] = [True if x == '1'.encode() else False for x in Dataset_phoneme['Class']]
Dataset_phoneme = Dataset_phoneme.drop(['Class'],axis=1)
Dataset_phoneme = Dataset_phoneme.drop(removeColumns(Dataset_phoneme),axis=1)
Dataset_phoneme = overSampling(underSampling(Dataset_phoneme))#Aplicando over e undersampling para 15% dos dados para tentar deixar mais proporcional o dataset
Dataset_phoneme.reset_index(inplace=True, drop=True)

3054 1586


In [17]:
Dataset_adult = InitDataset_adult.copy()
Dataset_adult['Target'] = [True if x == '<=50K'.encode() else False for x in Dataset_adult['class']]
Dataset_adult = Dataset_adult.drop(['class'],axis=1)
Dataset_adult['workclass'] = [x.decode('UTF-8') for x in Dataset_adult['workclass']]
Dataset_adult['education'] = [x.decode('UTF-8') for x in Dataset_adult['education']]
Dataset_adult['marital-status'] = [x.decode('UTF-8') for x in Dataset_adult['marital-status']]
Dataset_adult['occupation'] = [x.decode('UTF-8') for x in Dataset_adult['occupation']]
Dataset_adult['relationship'] = [x.decode('UTF-8') for x in Dataset_adult['relationship']]
Dataset_adult['race'] = [x.decode('UTF-8') for x in Dataset_adult['race']]
Dataset_adult['sex'] = [x.decode('UTF-8') for x in Dataset_adult['sex']]
Dataset_adult['native-country'] = [x.decode('UTF-8') for x in Dataset_adult['native-country']]

Dataset_adult = Dataset_adult.dropna() #Remoção das linhas com valores faltantes
Dataset_adult = get_dummies(Dataset_adult,'workclass')
Dataset_adult = get_dummies(Dataset_adult,'education')
Dataset_adult = get_dummies(Dataset_adult,'marital-status')
Dataset_adult = get_dummies(Dataset_adult,'occupation')
Dataset_adult = get_dummies(Dataset_adult,'relationship')
Dataset_adult = get_dummies(Dataset_adult,'race')
Dataset_adult = get_dummies(Dataset_adult,'sex')
Dataset_adult = get_dummies(Dataset_adult,'native-country')
Dataset_adult = Dataset_adult.drop(removeColumns(Dataset_adult),axis=1)

In [18]:
##Normalizando todos os dados
NOR_Dataset_PhishingWebsites = normalize(Dataset_PhishingWebsites)
NOR_Dataset_arrhythmia = normalize(Dataset_arrhythmia)
NOR_Dataset_Satellite = normalize(Dataset_Satellite)
NOR_Dataset_AedesSex = normalize(Dataset_AedesSex)
NOR_Dataset_airlines = normalize(Dataset_airlines)
NOR_Dataset_phoneme = normalize(Dataset_phoneme)
NOR_Dataset_adult = normalize(Dataset_adult)

In [ ]:
# knn(NOR_Dataset_PhishingWebsites) #93.8498643352427
knn(NOR_Dataset_arrhythmia) 
# knn(NOR_Dataset_Satellite) #99.2156862745098
# knn(NOR_Dataset_AedesSex) #98.73611111111111
# knn(NOR_Dataset_phoneme) #88.03945745992601
# knn(NOR_Dataset_adult) #81.71022998703337
# knn(NOR_Dataset_airlines) #81.71022998703337


# treeDecision(NOR_Dataset_PhishingWebsites)#96.02050045221586
# treeDecision(NOR_Dataset_arrhythmia)
# treeDecision(NOR_Dataset_Satellite)#99.08496732026144
# treeDecision(NOR_Dataset_AedesSex)#98.19444444444444
# treeDecision(NOR_Dataset_phoneme)#87.05302096177559
# treeDecision(NOR_Dataset_adult)#82.01733433426602
# treeDecision(NOR_Dataset_airlines)


# RForestDecision(NOR_Dataset_PhishingWebsites)#96.80434127223396
# RForestDecision(NOR_Dataset_arrhythmia)
# RForestDecision(NOR_Dataset_Satellite)#99.281045751634
# RForestDecision(NOR_Dataset_AedesSex)#98.38888888888889
# RForestDecision(NOR_Dataset_phoneme)#90.9987669543773
# RForestDecision(NOR_Dataset_adult)#85.68893741895857
# RForestDecision(NOR_Dataset_airlines)



In [19]:
##Separando os atributos dos datasets para testar qual o melhor algoritmo
PhishingWebsites_xTrain, PhishingWebsites_xTest, PhishingWebsites_yTrain, PhishingWebsites_yTest = train_test_split(NOR_Dataset_PhishingWebsites.drop(["Target"], axis=1), NOR_Dataset_PhishingWebsites["Target"], test_size=0.3)

Satellite_xTrain, Satellite_xTest, Satellite_yTrain, Satellite_yTest = train_test_split(NOR_Dataset_Satellite.drop(["Target"], axis=1), NOR_Dataset_Satellite["Target"], test_size=0.3)

AedesSex_xTrain, AedesSex_xTest, AedesSex_yTrain, AedesSex_yTest = train_test_split(NOR_Dataset_AedesSex.drop(["Target"], axis=1), NOR_Dataset_AedesSex["Target"], test_size=0.3)

phoneme_xTrain, phoneme_xTest, phoneme_yTrain, phoneme_yTest = train_test_split(NOR_Dataset_phoneme.drop(["Target"], axis=1), NOR_Dataset_phoneme["Target"], test_size=0.3)

adult_xTrain, adult_xTest, adult_yTrain, adult_yTest = train_test_split(NOR_Dataset_adult.drop(["Target"], axis=1), NOR_Dataset_adult["Target"], test_size=0.3)

airlines_xTrain, airlines_xTest, airlines_yTrain, airlines_yTest = train_test_split(NOR_Dataset_airlines.drop(["Target"], axis=1), NOR_Dataset_airlines["Target"], test_size=0.3)

arrhythmia_xTrain, arrhythmia_xTest, arrhythmia_yTrain, arrhythmia_yTest = train_test_split(NOR_Dataset_arrhythmia.drop(["Target"], axis=1), NOR_Dataset_arrhythmia["Target"], test_size=0.3)


In [26]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com o knn
CrossValidationDTPhishingWebsites,CrossValidationRFPhishingWebsites, CrossValidationKnnPhishingWebsites = crossVal(PhishingWebsites_xTest,PhishingWebsites_yTest,PhishingWebsites_xTrain,PhishingWebsites_yTrain) 
CrossValidationDTSatellite,CrossValidationRFSatellite, CrossValidationKnnSatellite = crossVal(Satellite_xTest,Satellite_yTest,Satellite_xTrain,Satellite_yTrain) 
CrossValidationDTAedesSex,CrossValidationRFAedesSex, CrossValidationKnnAedesSex = crossVal(AedesSex_xTest,AedesSex_yTest,AedesSex_xTrain,AedesSex_yTrain) 
CrossValidationDTPhoneme,CrossValidationRFPhoneme, CrossValidationKnnPhoneme = crossVal(phoneme_xTest,phoneme_yTest,phoneme_xTrain,phoneme_yTrain) 
CrossValidationDTAdult,CrossValidationRFAdult, CrossValidationKnnAdult = crossVal(adult_xTest,adult_yTest,adult_xTrain,adult_yTrain) 
CrossValidationDTArrhythmia,CrossValidationRFArrhythmia, CrossValidationKnnArrhythmia = crossVal(arrhythmia_xTest,arrhythmia_yTest,arrhythmia_xTrain,arrhythmia_yTrain) 

/home/lucas/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/lucas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/lucas/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
/home/lucas/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true

In [32]:
# print(CrossValidationDTPhishingWebsites.mean()*100,CrossValidationRFPhishingWebsites.mean()*100, CrossValidationKnnPhishingWebsites.mean()*100)
# print(CrossValidationDTSatellite.mean()*100,CrossValidationRFSatellite.mean()*100, CrossValidationKnnSatellite.mean()*100)
# print(CrossValidationDTAedesSex.mean()*100,CrossValidationRFAedesSex.mean()*100, CrossValidationKnnAedesSex.mean()*100)
# print(CrossValidationDTPhoneme.mean()*100,CrossValidationRFPhoneme.mean()*100, CrossValidationKnnPhoneme.mean()*100)
# print(CrossValidationDTAdult.mean()*100,CrossValidationRFAdult.mean()*100, CrossValidationKnnAdult.mean()*100)
print(CrossValidationDTArrhythmia.mean()*100,CrossValidationRFArrhythmia.mean()*100, CrossValidationKnnArrhythmia.mean()*100)


26.666666666666668 36.666666666666664 36.666666666666664


In [33]:
CrossValidationDTAirlines,CrossValidationRFAirlines, CrossValidationKnnAirlines = crossVal(airlines_xTest,airlines_yTest,airlines_xTrain,airlines_yTrain) 


Fazendo a analise de desempenho separadamente dos algoritmos utilizando f1-Score:

* Fórmula f1-Score

F1 = 2 * (precision * recall) / (precision + recall)
* PhishingWebsites: O melhor foi o random florest que teve um score melhor que os outros, ficando quase 1% melhor que o knn e 0.1% melhor que o Decision Tree. 
* Satellite: O melhor foi o random florest que teve um score melhor que os outros, ficando quase 21% melhor que o knn e 0.13% melhor que o Decision Tree. 
* AedesSex: O melhor foi o random florest que teve um score melhor que os outros, ficando quase 3% melhor que o knn e 0.4% melhor que o Decision Tree. 
* Phoneme: O melhor foi o random florest que teve um score melhor que os outros, ficando quase 3% melhor que o knn e 3% melhor que o Decision Tree. 
* Adult: O melhor foi o KNN que teve um score melhor que os outros, ficando quase 0.2% melhor que o random florest e 2% melhor que o Decision Tree. 
* Arrhythmia: Para este algoritmo todos ficaram extremamente ruins, e o knn e random florest ficaram com o mesmo score de 36.6.
* Airlines: No caso do airlines os scores também ficaram ruins,, no entanto o knn foi melhor que os outros, ficando com 47.98 de score, enquanto o Random florest ficou com 45.85 e o Decision tree 42.27.


In [34]:
print(CrossValidationDTAirlines.mean()*100,CrossValidationRFAirlines.mean()*100, CrossValidationKnnAirlines.mean()*100)


42.270785459424935 45.85076131956758 47.98148987475436


In [40]:
NArrayDT = np.array([])
NArrayDT = np.append(NArrayDT, CrossValidationDTPhishingWebsites)
NArrayDT = np.append(NArrayDT, CrossValidationDTSatellite)
NArrayDT = np.append(NArrayDT, CrossValidationDTAedesSex)
NArrayDT = np.append(NArrayDT, CrossValidationDTPhoneme)
NArrayDT = np.append(NArrayDT, CrossValidationDTAdult)
NArrayDT = np.append(NArrayDT, CrossValidationDTArrhythmia)
NArrayDT = np.append(NArrayDT, CrossValidationDTAirlines)

In [38]:
NArrayRF = np.array([])
NArrayRF = np.append(NArrayRF,CrossValidationRFPhishingWebsites)
NArrayRF = np.append(NArrayRF,CrossValidationRFSatellite)
NArrayRF = np.append(NArrayRF,CrossValidationRFAedesSex)
NArrayRF = np.append(NArrayRF,CrossValidationRFPhoneme)
NArrayRF = np.append(NArrayRF,CrossValidationRFAdult)
NArrayRF = np.append(NArrayRF,CrossValidationRFArrhythmia)
NArrayRF = np.append(NArrayRF,CrossValidationRFAirlines)

In [39]:
NArrayKnn = np.array([])
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnPhishingWebsites)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnSatellite)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnAedesSex)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnPhoneme)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnAdult)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnArrhythmia)
NArrayKnn = np.append(NArrayKnn,CrossValidationKnnAirlines)

In [42]:
print(NArrayDT.mean()*100,NArrayRF.mean()*100,NArrayKnn.mean()*100)

70.55488456804274 75.22450969655753 71.86325394350933


Como algoritmo mais genérico, o que teve o melhor score foi o Random florest com 75.22% e em segundo o knn.